In [1]:
import sqlalchemy as sa
import pandas as pd
import pyarrow as pa
import credentials_db as creds

In [2]:
pd.options.mode.copy_on_write = True

In [101]:
string_pa = pd.ArrowDtype(pa.string())

In [3]:
engine_sda = sa.create_engine('oracle+cx_oracle://'+creds.sda['usr']+':'+creds.sda['pwd']+creds.sda['host']+'/'+creds.sda['db'])

In [4]:
txt_for_sql = open('./sql/enrollment data fall census.sql','r').read()

In [5]:
queries = []
for i in range (19,25):
    queries.append(txt_for_sql.replace('__yr__',str(i)))
sql_enrolllment_data_fall = '\nunion\n'.join(queries)

In [63]:
enrollment = pd.read_sql(sql_enrolllment_data_fall,engine_sda,dtype={
        'sytdean_pidm':'uint32[pyarrow]',
        'sytdean_term_code_key':'category',
        'sytdean_registered_ind':'category',
        'sytdean_camp_code':'category',
        'sytdean_levl_code':'category',
        'sytdean_coll_code_1':'category',
        'sytdean_majr_code_1':'category',
        'sytdean_styp_code':'category',
        'sytdean_class_code':'category'
    })

In [205]:
def retention(df: pd.DataFrame) -> pd.DataFrame:
    def retention_rate(s: pd.Series) -> float:
        return (s=='Y').mean()
    return (df
        .assign(
            classification = df.sytdean_styp_code.case_when(caselist = [
                (df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_majr_code_1 == 'PHA6'), df.sytdean_styp_code.astype(string_pa) + '_PHA6'),
                (df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_majr_code_1 == 'NURS'), df.sytdean_styp_code.astype(string_pa) + '_BSN'),
                (df.sytdean_styp_code.isin(['F','T']), df.sytdean_styp_code),
                (df.sytdean_styp_code.isin(['C','I','R']) & (df.sytdean_majr_code_1 == 'PHA6'), df.sytdean_class_code.astype(string_pa) + '_PHA6'),
                (df.sytdean_styp_code.isin(['C','I','R']) & (df.sytdean_majr_code_1 == 'NURS'), df.sytdean_class_code.astype(string_pa) + '_BSN'),
                (df.sytdean_styp_code.isin(['C','I','R']), df.sytdean_class_code.astype(string_pa)),
                (df.sytdean_styp_code.isin(['N','S','V']), 'NSV')
            ]).astype('category'),
            retention_term = (df.sytdean_term_code_key.astype('int64[pyarrow]')+100).astype(string_pa).astype('category')
        )
        .merge(df,how='left',left_on=['sytdean_pidm','retention_term'],right_on=['sytdean_pidm','sytdean_term_code_key'])
        .pivot_table(index='classification', columns='retention_term', values='sytdean_registered_ind_y', aggfunc=retention_rate, dropna=False, observed=False)
        .loc[:,'202070':'202470']
        .pipe(lambda df_: df_.assign(
            avg_5yr = df_.mean(axis=1),
            avg_3yr = df_.loc[:,'202270':'202470'].mean(axis=1)
        ))
    )

In [206]:
retention(enrollment)

retention_term,202070,202170,202270,202370,202470,avg_5yr,avg_3yr
classification,,,,,,,
1Y,0.683486,0.633540,0.620000,0.638783,0.652720,0.645706,0.637168
1Y_PHA6,0.818182,0.428571,1.000000,0.777778,1.000000,0.804906,0.925926
2Y,0.885101,0.875944,0.865741,0.891827,0.873121,0.878347,0.876896
2Y_BSN,NaN,NaN,NaN,NaN,0.866667,0.866667,0.866667
2Y_PHA6,0.977273,0.971074,0.890110,0.944134,0.930636,0.942645,0.921627
3Y,0.810780,0.795310,0.741512,0.681529,0.738983,0.753623,0.720674
3Y_BSN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000
3Y_PHA6,0.990099,0.977778,0.957265,0.960265,0.972067,0.971495,0.963199
4Y,0.046757,0.046720,0.053855,0.050430,0.043027,0.048158,0.049104
